# 04. ピッチ検出

音の基本周波数（ピッチ）を検出する様々な手法を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import correlate, find_peaks
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 自己相関によるピッチ検出

In [ ]:
def autocorrelation_pitch_detection(signal, sample_rate, min_freq=80, max_freq=800):
    """
    自己相関によるピッチ検出
    """
    # 自己相関の計算
    autocorr = correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr)//2:]
    
    # 正規化
    autocorr = autocorr / autocorr[0]
    
    # ピッチ範囲に対応するラグ範囲
    min_lag = int(sample_rate / max_freq)
    max_lag = int(sample_rate / min_freq)
    
    # 指定範囲でピークを検索
    search_autocorr = autocorr[min_lag:max_lag]
    peaks, _ = find_peaks(search_autocorr, height=0.3)
    
    if len(peaks) > 0:
        # 最大のピークを選択
        best_peak_idx = peaks[np.argmax(search_autocorr[peaks])]
        lag = best_peak_idx + min_lag
        pitch = sample_rate / lag
        confidence = search_autocorr[best_peak_idx]
    else:
        pitch = 0
        confidence = 0
        lag = 0
    
    return pitch, confidence, autocorr, lag

# テスト信号の生成
sample_rate = 8000
duration = 1.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# 異なるピッチの信号をテスト
test_pitches = [110, 220, 330, 440]  # Hz
noise_levels = [0, 0.1, 0.3, 0.5]

plt.figure(figsize=(16, 12))

for i, (pitch, noise_level) in enumerate(zip(test_pitches, noise_levels)):
    # 倍音を含む信号の生成
    signal = (np.sin(2 * np.pi * pitch * t) + 
             0.5 * np.sin(2 * np.pi * pitch * 2 * t) +
             0.3 * np.sin(2 * np.pi * pitch * 3 * t))
    
    # ノイズを追加
    if noise_level > 0:
        np.random.seed(42 + i)
        signal += noise_level * np.random.normal(0, 1, len(signal))
    
    # ピッチ検出
    detected_pitch, confidence, autocorr, lag = autocorrelation_pitch_detection(
        signal, sample_rate)
    
    # 信号の表示
    plt.subplot(4, 3, i*3 + 1)
    plt.plot(t[:400], signal[:400], 'b-', linewidth=2)
    plt.title(f'信号: {pitch} Hz (ノイズ: {noise_level})')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # 自己相関関数
    plt.subplot(4, 3, i*3 + 2)
    lag_axis = np.arange(len(autocorr)) / sample_rate * 1000  # ms
    plt.plot(lag_axis[:1000], autocorr[:1000], 'g-', linewidth=2)
    
    if detected_pitch > 0:
        period_ms = 1000 / detected_pitch
        plt.axvline(x=period_ms, color='r', linestyle='--', 
                   label=f'検出周期: {period_ms:.1f} ms')
    
    plt.title('自己相関関数')
    plt.xlabel('ラグ (ms)')
    plt.ylabel('相関係数')
    plt.legend()
    plt.grid(True)
    
    # 結果まとめ
    plt.subplot(4, 3, i*3 + 3)
    plt.text(0.1, 0.8, f'真のピッチ: {pitch} Hz', fontsize=12, 
            transform=plt.gca().transAxes)
    plt.text(0.1, 0.6, f'検出ピッチ: {detected_pitch:.1f} Hz', fontsize=12,
            transform=plt.gca().transAxes, color='red')
    plt.text(0.1, 0.4, f'誤差: {abs(detected_pitch - pitch):.1f} Hz', fontsize=12,
            transform=plt.gca().transAxes)
    plt.text(0.1, 0.2, f'信頼度: {confidence:.2f}', fontsize=12,
            transform=plt.gca().transAxes)
    
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.title('検出結果')
    plt.axis('off')

plt.tight_layout()
plt.show()

print("自己相関法の特徴:")
print("- 周期性のある信号に対して有効")
print("- 倍音があっても基音を検出可能")
print("- ノイズに対してある程度頑健")
print("- 計算コストが比較的低い")

## 2. FFTベースのピッチ検出

In [ ]:
def fft_pitch_detection(signal, sample_rate):
    """
    FFTベースのピッチ検出（倍音パターン認識）
    """
    # FFT
    fft_result = fft(signal)
    frequencies = fftfreq(len(signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    
    freq_positive = frequencies[positive_mask]
    amplitude_positive = np.abs(fft_result[positive_mask])
    
    # ピーク検出
    peaks, _ = find_peaks(amplitude_positive, 
                         height=np.max(amplitude_positive) * 0.1,
                         distance=10)
    
    peak_frequencies = freq_positive[peaks]
    peak_amplitudes = amplitude_positive[peaks]
    
    # 基音候補を探索
    best_fundamental = 0
    best_score = 0
    
    for candidate_freq in peak_frequencies:
        if candidate_freq < 50 or candidate_freq > 800:  # 範囲外は除外
            continue
        
        # この候補が基音だと仮定して倍音のスコアを計算
        score = 0
        harmonic_count = 0
        
        for harmonic in range(1, 8):  # 7倍音まで
            target_freq = candidate_freq * harmonic
            if target_freq > sample_rate / 2:
                break
            
            # 最も近いピークを探す
            distances = np.abs(peak_frequencies - target_freq)
            closest_idx = np.argmin(distances)
            
            # 許容誤差内にピークがあるか
            if distances[closest_idx] < candidate_freq * 0.1:  # 10%の誤差許容
                # 振幅で重み付けしたスコア
                weight = peak_amplitudes[closest_idx] / np.max(peak_amplitudes)
                score += weight / harmonic  # 高次倍音は重みを下げる
                harmonic_count += 1
        
        # 倍音の数も考慮
        score *= (harmonic_count / 7)  # 正規化
        
        if score > best_score:
            best_score = score
            best_fundamental = candidate_freq
    
    return best_fundamental, best_score, freq_positive, amplitude_positive, peak_frequencies, peak_amplitudes

# FFT法とHPS（Harmonic Product Spectrum）法の比較
def harmonic_product_spectrum(signal, sample_rate, num_harmonics=5):
    """
    ハーモニック積スペクトル法
    """
    # FFT
    fft_result = fft(signal)
    frequencies = fftfreq(len(signal), 1/sample_rate)
    positive_mask = frequencies >= 0
    
    freq_positive = frequencies[positive_mask]
    amplitude_positive = np.abs(fft_result[positive_mask])
    
    # HPS計算
    hps = amplitude_positive.copy()
    
    for harmonic in range(2, num_harmonics + 1):
        # スペクトルをダウンサンプリング
        downsampled_length = len(amplitude_positive) // harmonic
        downsampled = amplitude_positive[:downsampled_length * harmonic:harmonic]
        
        # 積を計算
        hps[:len(downsampled)] *= downsampled
    
    # ピッチ検出
    # 低周波数ノイズを除去
    min_freq_idx = np.where(freq_positive >= 50)[0]
    if len(min_freq_idx) > 0:
        search_start = min_freq_idx[0]
        max_freq_idx = np.where(freq_positive <= 800)[0]
        search_end = max_freq_idx[-1] if len(max_freq_idx) > 0 else len(hps)
        
        search_hps = hps[search_start:search_end]
        peak_idx = np.argmax(search_hps) + search_start
        detected_pitch = freq_positive[peak_idx]
    else:
        detected_pitch = 0
    
    return detected_pitch, hps, freq_positive, amplitude_positive

# 複雑な音響信号でのテスト
complex_signals = {
    '基音+倍音': lambda f, t: (np.sin(2*np.pi*f*t) + 0.5*np.sin(2*np.pi*f*2*t) + 0.3*np.sin(2*np.pi*f*3*t)),
    '欠損基音': lambda f, t: (0.6*np.sin(2*np.pi*f*2*t) + 0.4*np.sin(2*np.pi*f*3*t) + 0.3*np.sin(2*np.pi*f*4*t)),
    '不協和音': lambda f, t: (np.sin(2*np.pi*f*t) + np.sin(2*np.pi*(f*1.1)*t)),
    'コード': lambda f, t: (np.sin(2*np.pi*f*t) + np.sin(2*np.pi*(f*5/4)*t) + np.sin(2*np.pi*(f*3/2)*t))
}

test_frequency = 200  # Hz

plt.figure(figsize=(16, 16))

for i, (signal_name, signal_func) in enumerate(complex_signals.items()):
    # 信号生成
    signal = signal_func(test_frequency, t)
    
    # 各手法でピッチ検出
    autocorr_pitch, autocorr_conf, _, _ = autocorrelation_pitch_detection(signal, sample_rate)
    fft_pitch, fft_score, freq_pos, amp_pos, peak_freqs, peak_amps = fft_pitch_detection(signal, sample_rate)
    hps_pitch, hps, freq_hps, amp_hps = harmonic_product_spectrum(signal, sample_rate)
    
    # 時間波形
    plt.subplot(4, 4, i*4 + 1)
    plt.plot(t[:400], signal[:400], 'b-', linewidth=2)
    plt.title(f'{signal_name}（時間波形）')
    plt.ylabel('振幅')
    plt.grid(True)
    
    # FFTスペクトル
    plt.subplot(4, 4, i*4 + 2)
    plt.plot(freq_pos, amp_pos, 'b-', alpha=0.7, linewidth=1)
    plt.plot(peak_freqs, peak_amps, 'ro', markersize=6, alpha=0.8)
    plt.axvline(x=fft_pitch, color='red', linestyle='--', label=f'FFT: {fft_pitch:.1f} Hz')
    plt.title('FFTスペクトルとピーク')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 1000)
    
    # HPSスペクトル
    plt.subplot(4, 4, i*4 + 3)
    plt.plot(freq_hps, amp_hps, 'b-', alpha=0.7, label='元スペクトル')
    plt.plot(freq_hps, hps / np.max(hps) * np.max(amp_hps), 'g-', linewidth=2, label='HPS')
    plt.axvline(x=hps_pitch, color='green', linestyle='--', label=f'HPS: {hps_pitch:.1f} Hz')
    plt.title('Harmonic Product Spectrum')
    plt.ylabel('振幅')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 1000)
    
    # 結果比較
    plt.subplot(4, 4, i*4 + 4)
    methods = ['真値', '自己相関', 'FFT', 'HPS']
    pitches = [test_frequency, autocorr_pitch, fft_pitch, hps_pitch]
    colors = ['black', 'blue', 'red', 'green']
    
    bars = plt.bar(methods, pitches, color=colors, alpha=0.7)
    plt.axhline(y=test_frequency, color='black', linestyle='--', alpha=0.5)
    plt.title('ピッチ検出結果比較')
    plt.ylabel('周波数 (Hz)')
    plt.xticks(rotation=45)
    
    # 数値を表示
    for bar, pitch in zip(bars, pitches):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                f'{pitch:.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("各手法の特徴:")
print("1. 自己相関法: 周期性を利用、倍音に強い")
print("2. FFT法: 倍音パターン認識、複雑な音に対応")
print("3. HPS法: 基音が欠損していても検出可能")

## 練習問題

1. 楽器の音を録音して、異なるピッチ検出法の精度を比較してみましょう
2. ノイズの多い環境での各手法の頑健性を評価してみましょう
3. リアルタイムピッチ検出システムを実装してみましょう